#**Text to Image Generation**

###**What Happens Step by Step?**
1. User Input  → Get text from the user to generate image
2. API Call → OpenAI DALL·E API returns a URL (not an image).
3. Download Image → requests.get(url).content fetches the image data.
4. Convert to Image → BytesIO turns the binary data into a format PIL can read.
5. Display in Gradio → The PIL image is returned and shown in the UI.


###**Install all dependencies**

In [1]:
!pip install openai gradio requests pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00


## **Application**

###**Downloads the `hrdataset.zip` file from the CloudYuga GitHub repo**

Saves it in the current working directory of notebook

(e.g., /content/ in Google Colab).


In [2]:
!wget https://github.com/cloudyuga/mastering-genai-w-python/raw/refs/heads/main/hrdataset.zip

--2025-05-22 07:28:43--  https://github.com/cloudyuga/mastering-genai-w-python/raw/refs/heads/main/hrdataset.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cloudyuga/mastering-genai-w-python/refs/heads/main/hrdataset.zip [following]
--2025-05-22 07:28:43--  https://raw.githubusercontent.com/cloudyuga/mastering-genai-w-python/refs/heads/main/hrdataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9530 (9.3K) [application/zip]
Saving to: ‘hrdataset.zip’

hrdataset.zip       100%[===================>]   9.31K  --.-KB/s    in 0s      

2025-05-22 07:28:43 (30.4 MB/s) - ‘hrdataset.zip’ sav

###**Unzip `hrdataset.zip` file**
- It will automatically create **`hrdataset`** folder in our current working directory (/content/ in Google Colab)

In [3]:
!unzip hrdataset.zip

Archive:  hrdataset.zip
   creating: hrdataset/
   creating: hrdataset/policies/
  inflating: hrdataset/policies/leave_policies.md  
  inflating: hrdataset/policies/training_and_development.md  
  inflating: hrdataset/policies/employee_benefits.md  
  inflating: hrdataset/policies/holiday_calendar.md  
  inflating: hrdataset/policies/events_calendar.md  
   creating: hrdataset/surveys/
  inflating: hrdataset/surveys/Employee_Culture_Survey_Responses.csv  
   creating: hrdataset/employees/
  inflating: hrdataset/employees/108_Rajesh_Kulkarni.md  
  inflating: hrdataset/employees/106_Neha_Malhotra.md  
  inflating: hrdataset/employees/103_Anjali_Das.md  
  inflating: hrdataset/employees/105_Sunita_Patil.md  
  inflating: hrdataset/employees/101_Priya_Sharma.md  
  inflating: hrdataset/employees/102_Rohit_Mehra.md  
  inflating: hrdataset/employees/104_Karan_Kapoor.md  
  inflating: hrdataset/employees/109_Meera_Iyer.md  
  inflating: hrdataset/employees/110_Aditya_Jain.md  
  inflating: 

In [4]:
employee_files_path = "hrdataset/employees"
policy_files_path = "hrdataset/policies"


###**Retrive API key from Secrets and Set as an ENV**

In [5]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [6]:
# Set OPENAI_API_KEY as an ENV
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [7]:
import gradio as gr
from openai import OpenAI

In [8]:
client = OpenAI()

In [9]:
import re

# Function to read and extract details from Markdown files
def read_markdown_files(directory):
    """Read and parse content from all Markdown files in a directory."""
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".md"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read()

            # Extract details using regex
            employee_data = {
                "filename": filename,
                "Employee ID": re.search(r"Employee ID:\s*(\d+)", content),
                "Name": re.search(r"Name:\s*([\w\s]+)", content),
                "Role": re.search(r"Role:\s*([\w\s]+)", content),
                "Department": re.search(r"Department:\s*([\w\s]+)", content),
                "DOB": re.search(r"Date of Birth:\s*([\d-]+)", content),
                "Hobbies": re.search(r"Hobbies:\s*([\w\s,]+)", content)  # Extract hobbies
            }

            # Convert regex matches to actual values
            for key, match in employee_data.items():
                if match and hasattr(match, 'group'):  # Check if 'match' has 'group' attribute
                    employee_data[key] = match.group(1)  # Get the matched group value
                else:
                    employee_data[key] = None  # If no match or no 'group' method, set it to None

            documents.append(employee_data)

    return documents

In [10]:
# Load employee data from markdown files
employee_docs = read_markdown_files(employee_files_path)

In [11]:
# Function to extract Name and Role from the user's input using LLM
def extract_name_and_role(input_text):
    """Use LLM to extract Name and Role from user input."""
    try:
        response = client.Completion.create(
            model="text-davinci-003",
            prompt=f"Extract the Name and Role from this input: '{input_text}'.",
            max_tokens=100,
            temperature=0.3
        )
        extracted_text = response.choices[0].text.strip()
        # Extract Name and Role from the model's response
        name = re.search(r"Name: ([\w\s]+)", extracted_text)
        role = re.search(r"Role: ([\w\s]+)", extracted_text)

        return name.group(1) if name else None, role.group(1) if role else None
    except Exception as e:
        return None, None


In [12]:
# Function to fetch employee details from parsed docs
def get_employee_details(input_text, all_docs):
    input_text = input_text.lower()

    # Extract Name and Role from the input using LLM
    name, role = extract_name_and_role(input_text)

    # Search for employee details based on Name or Role
    for doc in all_docs:
        name_match = name and name.lower() in doc.get("Name", "").lower()
        role_match = role and role.lower() in doc.get("Role", "").lower()

        if name_match or role_match:
            return doc  # Return the matched employee details

    return None  # No match found


In [13]:
# Function to generate a professional birthday card
def generate_birthday_card(input_text):
    employee = get_employee_details(input_text, employee_docs)

    if employee:
        name = employee.get("Name", "Valued Employee")
        role = employee.get("Role", "Team Member")
        dob = employee.get("DOB", "N/A")
        hobbies = employee.get("Hobbies", "Enjoying life")  # Default if no hobby found

        # Crafting a professional, personalized birthday card prompt
        prompt = (
            f"A professional birthday card for {name}, our esteemed {role}. "
            f"Include the following details: Date of Birth: {dob}. "
            f"The card should be elegant and warm, with design elements related to {name}'s hobbies: {hobbies}. "
            f"Ensure the name {name} is clearly visible on the card."
        )
    else:
        prompt = f"A birthday card with vibrant colors and an artistic theme for {input_text}."

    try:
        # Generate image using DALL·E
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            n=1
        )
        return response.data[0].url  # Return the generated image URL

    except Exception as e:
        return f"Error: {str(e)}"

In [14]:
# Gradio Interface for generating birthday cards
iface = gr.Interface(
    fn=generate_birthday_card,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Enter request (e.g., 'Generate a birthday card for CEO')",
        label="Card Request"
    ),
    outputs=gr.Image(label="Generated Card"),
    title="AI-Generated Birthday Card",
    description="Enter an employee name or role to generate a personalized birthday card."
)

# Launch Gradio App


In [15]:
if __name__ == "__main__":
    iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://72a567400165d82711.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
